In [1]:
with open("src/grammar/interp.gram") as gramfile:
    flines = [line.strip() for line in gramfile]

In [2]:
groups: list[str] = []
cur_group = ""

for line in flines:
    if "::=" in line and cur_group:
        groups.append(cur_group)
        cur_group = line
    else:
        cur_group = cur_group + line

groups.append(cur_group)

In [3]:
def split_strip(s: str, sep: str) -> list[str]:
    return [x.strip() for x in s.split(sep)]

In [4]:
delta: dict[str, list[tuple[str, str]]] = {}
ops: set[str] = set()

for group in groups:
    symbol, rhs_str = split_strip(group, "::=")
    
    if symbol == "OPS":
        ops.update(rhs_str.split())
    else:
        delta[symbol] = []
        rhs_list = split_strip(rhs_str, "\\")
        for item in rhs_list:
            cats, act = split_strip(item, "___")
            act = act[1:-1].strip()
            delta[symbol].append((cats, act))

In [6]:
def parse_rule(rule: str) -> list[tuple[str, str | list[str]]]:
    parsed: list[tuple[str, str | list[str]]] = []

    in_opt = False
    in_star = False
    cur_group: list[str] = []

    for item in rule.split():
        if item == '[':
            in_star = True
        elif item == ']':
            in_star = False
            parsed.append(('star', cur_group))
            cur_group = []
        elif item == '(':
            in_opt = True
        elif item == ')':
            in_opt = False
            parsed.append(('opt', cur_group))
            cur_group = []
        elif in_opt or in_star: cur_group.append(item)
        else:
            parsed.append(('symbol', item))
    
    return parsed